# 🔐 Demo: Key Vault + .env Fallback & Event Hub Producer (Market Data)

Este notebook demonstra:
1. Como carregar segredos do **Azure Key Vault** com *fallback* para `.env`.
2. Como ler a lista de símbolos em `assets/sample_sintetic_data/market_data_subscriptions.json`.
3. Como enviar eventos para o **Event Hub** `market-data` usando `partition_key = símbolo`.
4. Boas práticas de masking de segredos.

> ⚠️ Este notebook é para demonstração / desenvolvimento. Não exponha segredos reais em logs.

## ✅ Pré-requisitos
- Infra provisionada (Bicep ou Terraform) com Key Vault e Event Hub.
- Segredos criados: `postgres-admin-password`, `eventhub-connection-string` (se Terraform) ou criado manualmente.
- Variável de ambiente `KEY_VAULT_URI` exportada OU linha correspondente no `.env`.
- Permissão de acesso (Get/List) ao Key Vault via Managed Identity ou Azure CLI login.

### Pacotes Python necessários
| Pacote | Uso |
|--------|-----|
| `python-dotenv` | Carregar `.env` local |
| `azure-identity` | Autenticação unificada |
| `azure-keyvault-secrets` | Ler segredos |
| `azure-eventhub` | Enviar eventos |

Se faltarem, instale no cluster / ambiente de execução.

In [ ]:
# (Opcional) Instalar dependências - descomente se necessário
# %pip install python-dotenv azure-identity azure-keyvault-secrets azure-eventhub --quiet
import os, json, time, uuid
from pathlib import Path
from typing import Optional

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.eventhub import EventHubProducerClient, EventData

print('Pacotes importados.')

In [ ]:
# Carrega .env (fallback).
load_dotenv()
KEY_VAULT_URI = os.getenv('KEY_VAULT_URI')
print('KEY_VAULT_URI definido?' , bool(KEY_VAULT_URI))

In [ ]:
# Utilitário para obter segredos: tenta Key Vault, senão .env
class SecretProvider:
    def __init__(self, kv_uri: Optional[str]):
        self.kv_uri = kv_uri
        self._client = None
        if kv_uri:
            try:
                cred = DefaultAzureCredential(exclude_interactive_browser_credential=False)
                self._client = SecretClient(vault_url=kv_uri, credential=cred)
            except Exception as ex:  # noqa
                print(f'[WARN] Falha iniciando Key Vault client: {ex}')
                self._client = None
    def get(self, name: str, env_fallback: Optional[str] = None) -> str:
        # 1. Key Vault
        if self._client is not None:
            try:
                return self._client.get_secret(name).value
            except Exception as ex:  # noqa
                print(f'[WARN] Falha lendo {name} no KV: {ex}')
        # 2. Fallback env/.env
        candidates = [env_fallback, name, name.upper().replace('-', '_')]
        for key in candidates:
            if not key: continue
            val = os.getenv(key)
            if val: return val
        raise RuntimeError(f'Segredo {name} não encontrado em KV nem env')

sp = SecretProvider(KEY_VAULT_URI)
POSTGRES_PASSWORD = sp.get('postgres-admin-password', env_fallback='PGPASSWORD')
try:
    EVENTHUB_CONNECTION = sp.get('eventhub-connection-string', env_fallback='EVENTHUB_CONNECTION_STRING')
except Exception as ex:  # noqa
    EVENTHUB_CONNECTION = None
    print('[INFO] Segredo eventhub-connection-string não disponível - pulando envio de eventos.')
def mask(value: Optional[str], visible: int = 4) -> str:
    if not value: return '<none>'
    return value[:visible] + '***' + value[-visible:] if len(value) > visible*2 else '***'
print('POSTGRES_PASSWORD (masked):', mask(POSTGRES_PASSWORD))
print('EVENTHUB_CONNECTION (masked):', mask(EVENTHUB_CONNECTION))

## 📄 Carregar lista de símbolos
Arquivo: `assets/sample_sintetic_data/market_data_subscriptions.json` contendo grupos e lista unificada (`all`).

In [ ]:
SYMBOLS_FILE = Path('..') / 'assets' / 'sample_sintetic_data' / 'market_data_subscriptions.json'
data = json.loads(SYMBOLS_FILE.read_text(encoding='utf-8'))
symbols = data['all'][:20]  # reduzido para demo
print(f'Total símbolos (all): {len(data[
])}, usaremos {len(symbols)} nesta demo.')
symbols[:10]

## 🚀 Produção de Eventos (Partition Key = símbolo)
Cada evento será particionado determinísticamente pelo ticker, garantindo ordenação per-partition.
Se Capture estiver habilitado, arquivos Avro serão gravados no container `ehcapture` conforme janela/tamanho configurados.

In [ ]:
if EVENTHUB_CONNECTION:
    producer = EventHubProducerClient.from_connection_string(conn_str=EVENTHUB_CONNECTION)
    batch = None
    sent = 0
    with producer:
        for sym in symbols:
            payload = {
                'id': str(uuid.uuid4()),
                'symbol': sym,
                'price': round(100 + 50 * (hash(sym) % 100) / 100, 2),
                'ts': time.time()
            }
            ed = EventData(json.dumps(payload))
            # Para garantir direcionamento por ticker use send_event_data_batch com partition_key
            producer.send_batch([ed], partition_key=sym)
            sent += 1
    print(f'Enviados {sent} eventos para o Event Hub market-data.')
else:
    print('EVENTHUB_CONNECTION ausente - simulação somente.')

## 🧪 Próximos Passos
1. Criar um consumidor (Databricks Structured Streaming / Spark).
2. Escrever Delta Bronze com schema evolving.
3. Aplicar transformações Silver/Gold.
4. Reprocessar histórico via arquivos de Capture se necessário.

> Dica: para replays, liste blobs no container `ehcapture` e reprocesse Avro em ordem de tempo.

## ✅ Conclusão
Você viu como: (a) resolver segredos via Key Vault com fallback, (b) carregar lista de símbolos, (c) enviar eventos particionados por ticker.
A próxima etapa é automatizar pipelines de ingestão e lineage no Purview.